In [55]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
summoner_name = "야식은치킨이지"
match = pd.read_json("data/{name}/match.json".format(name=summoner_name))
timeline = pd.read_json("data/{name}/timeline.json".format(name=summoner_name))

In [57]:
# match 데이터를 저장할 때 index 부분을 조정해주어야 할 듯 
for i in range(100):
    match.iloc[i,0] = (match.iloc[i,0]-1)%10+1

match.head()

,index,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,...,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,perks.statPerks.defense,perks.statPerks.flex,perks.statPerks.offense,perks.styles
0,6,13,0,0,12602,14,31,Chogath,0,2,...,0,0,0,0,0,False,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':..."
1,5,3,0,0,14804,16,14,Sion,0,3,...,0,19,2,4,4,False,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':..."
2,10,2,0,1,8051,11,21,MissFortune,0,4,...,0,11,2,3,8,False,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':..."
3,3,4,0,0,14620,15,51,Caitlyn,0,6,...,0,27,4,3,16,False,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':..."
4,5,2,0,0,7543,11,150,Gnar,0,3,...,0,8,2,0,4,False,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':..."


# 경기 수 관련

In [58]:
match_length = len(timeline)
match_invade = 0

# 이벤트 분석

In [59]:
match_index = 1
match_time_length = len(timeline.iloc[match_index].info['frames'])
team_id = match.teamId[match_index]
win = match.win[match_index]

events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

In [62]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    team_id = match.teamId[match_index]
    win = match.win[match_index]
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    for t in range(1,3):  # 1 ~ 2 분 사이
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1
                # display(pd.json_normalize(e))

    if is_invade[match_index]:
        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        if team_id == 100:
            if win: invade_win_team[match_index] = 100
            else: invade_win_team[match_index] = 200
        else:
            if win: invade_win_team[match_index] = 200
            else: invade_win_team[match_index] = 100    
        print("--------------------------------------------------")
        print("invade detected: %f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% "blue" if invade_win_team[match_index] == 100 else "red")

--------------------------------------------------
invade detected: 0.740417
< kills >
  blue : 4 , red : 1
< total_gold_3_min >
  blue: 13290 , red: 11933
< gold_gap >
  1357
< win >
  blue
--------------------------------------------------
invade detected: 0.824783
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 4258 , red: 4764
< gold_gap >
  -506
red
--------------------------------------------------
invade detected: 0.767733
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 4571 , red: 4750
< gold_gap >
  -179
< win >
  blue
--------------------------------------------------
invade detected: 0.987433
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 7404 , red: 8884
< gold_gap >
  -1480
red
--------------------------------------------------
invade detected: 1.430283
< kills >
  blue : 2 , red : 1
< total_gold_3_min >
  blue: 12770 , red: 11884
< gold_gap >
  886
< win >
  blue
--------------------------------------------------
invade detected: 1.823

In [61]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap
0,True,44425,4,1,100,13290,11933,1357
10,True,85817,2,1,100,12770,11884,886
11,True,109439,2,1,100,12470,11800,670
12,True,73810,4,1,100,13207,11693,1514
17,True,67085,1,0,200,9007,7358,1649
25,True,85660,1,0,200,8381,7817,564
30,True,119040,1,0,200,8404,7791,613
31,True,117294,1,0,100,8681,8270,411
32,True,58295,2,0,100,10104,6965,3139
42,True,80851,2,1,100,13726,12922,804


0.71%


In [63]:
match_invade

45